In [1]:
import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

In [2]:
from googleapiclient.discovery import build
import os

GOOGLE_CSE_ID = os.environ.get("GOOGLE_CSE_ID")
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

def get_google_search(query):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    request = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=3)
    response = request.execute()
    results = response.get("items", [])
    if not results:
        return [{"Result": "No good Google Search Result was found"}]
    metadata_results = [get_metadata_result(result) for result in results]
    return metadata_results

def get_metadata_result(result):
    metadata_result = {
        "title": result["title"],
        "link": result["link"],
    }
    if "snippet" in result:
        metadata_result["snippet"] = result["snippet"]
    return metadata_result

In [3]:
llm_config = {
    "config_list": config_list_gpt4,
    "seed": 42,
    "functions": [
        {
            "name": "get_google_search",
            "description": "Get Google Search Results",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search Query"}
                },
                "required": ["query"],
            },
        }
    ],
}
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the product manager to discuss the plan. Reply TERMINATE if the task has been solved. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.",
    human_input_mode="ALWAYS",
)
executor = autogen.UserProxyAgent(
    name="Executor",
    human_input_mode="NEVER",
    system_message='Executor. Execute the function call by others and report the result.',
    function_map={"get_google_search": get_google_search},
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, executor, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [4]:
user_proxy.initiate_chat(manager, message="Learn about AI Agent and AITable, find AI Agent's potential applications in AITable.")

Admin (to chat_manager):

Learn about AI Agent and AITable, find AI Agent's potential applications in AITable.

--------------------------------------------------------------------------------
Product_manager (to chat_manager):

AI Agent is a type of artificial intelligence that learns from the environment by observing and interacting with it, and takes actions to achieve certain goals. These agents can be autonomous, like a self-driving car or a drone, or they can assist users in accomplishing tasks, like a voice assistant or a chatbot. 

AITable, on the other hand, is a hypothetical or conceptual tool that utilizes AI for advanced data management and analysis. It is similar to how we use traditional tables or spreadsheets, but with integrated AI capabilities such as machine learning, natural language processing, predictive analytics, etc.

To integrate AI agents with AITables, we can potentially develop a software product in multiple industries. For example:

1. Predictive AI Assista